<img style="float: right;" src="pics\BIC_128.png" width="300" />

# Pandas

- Python package that is essentially a souped-up Excel
- Built off numpy, so you will see a lot of similarity
- Adds **labels** to data for easy readability
- Adds an analog of R data frames

According to their website, pandas is good for:
* Tabular data with heterogeneously-typed columns, as in an SQL table or Excel spreadsheet
* Ordered and unordered time series data
* Arbitrary matrix data (homogeneously typed or heterogeneous) with row and column labels (e.g. correlation matrix)
* Any other form of observational / statistical data sets. The data need not be labeled at all to be placed into a pandas data structure

Pandas handles the following attributes/actions well when dealing with tabular/matrix data:
* Easy handling of missing data (represented as NaN) in floating point as well as non-floating point data
* Size mutability: columns can be inserted and deleted from DataFrame
* Automatic and explicit data alignment: objects can be explicitly aligned to a set of labels, or the user can simply ignore the labels and let Series, DataFrame, etc. automatically align the data for you in computations
* Powerful, flexible group by functionality to perform split-apply-combine operations on data sets, for both aggregating and transforming data
* Intelligent label-based slicing, fancy indexing, and subsetting of large data sets
* Intuitive merging and joining data sets
* Robust IO tools for loading data from flat files (CSV and delimited), Excel files, databases, and saving / loading data from the ultrafast HDF5 format
* Time series-specific functionality: date range generation and frequency conversion, moving window statistics, date shifting, and lagging.



In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Contents
- [Series](#series)
- [DataFrames](#dataframes)
- [Pandas Index](#index)
- [Hierarchical index](#hierarchical)
- [Missing data](#missing)
- [`groupby`](#group)
- [Merging, joining, concatenating](#combine)

___
<a id='series'></a>
## Series
- "One-dimensional ndarray with axis labels", Pandas
- Time series are fit well for this
- Individual data points are labelled and reffered to by the Series's **Index**

### Creating a Series

- `Series` function can create Series from lists, arrays, dictionaries, and many other Python objects

In [ ]:
# Calcium imaging data
ca_data = np.loadtxt('sample_data/ca-traces.txt', delimiter=',')

In [ ]:
ca_data[0, :]

In [ ]:
plt.plot(ca_data[0, :])

In [ ]:
neuron0 = pd.Series(data=ca_data[0, :])
neuron0

You can add labels for indices.

In [ ]:
ts = np.arange(0, ca_data.shape[1]) * 0.2
neuron0 = pd.Series(data=ca_data[0, :], index=ts)
neuron0.head()

In [ ]:
plt.plot(neuron0)

Dictionaries already have labels supplied!

In [ ]:
genotype = {
    'mouse2': 'Cre+',
    'mouse5': 'Cre-',
    'mouse6': 'Cre-',
    'mouse9': 'Cre+',
}

In [ ]:
pd.Series(genotype)

Note that any object can be provided, not just numbers. Here we provided strings, but we could also provide any other type of object.

### Using an Index

- Defined by an ndarray of the same size as the series
- Used as a guide for operations

In [ ]:
np1 = pd.Series([24, 20, 55, 32, 100], index=['mouse5', 'mouse6', 'mouse9', 'mouse2', 'mouse10'])

np2 = pd.Series([20, 20, 33, 51], index=['mouse5', 'mouse6', 'mouse2', 'mouse91'])

In [ ]:
np1

In [ ]:
np2.size

In [ ]:
np1['mouse9']

In [ ]:
# this won't work. Why?
np2['mouse9']

In [ ]:
np1.add(np2, fill_value=0)

---
<a id='dataframes'></a>
## DataFrames

- 2-dimensional Pandas objects (as opposed to 1-dimensional Series)
- Essentially a column stack of Series
- Similar to data frames of R

In [ ]:
pd.DataFrame(ca_data).head()

Add labels to really take advantage of Pandas

In [ ]:
neuron_name = ['neuron{}'.format(n) for n in range(ca_data.shape[0])]
df_ca = pd.DataFrame(ca_data.T, index=ts, columns=neuron_name)
df_ca.tail()

You can give your columns and indices names for convenience.

In [ ]:
df_ca.columns.name

In [ ]:
df_ca.columns.name = 'neuron'
df_ca.index.name = 'time'
df_ca

### Selection and Indexing

- Choose columns with brackets `[]` we are used to (unless you use boolean mask, see [conditional selection](#conditional))
- `loc` method allows you to choose index then, optionally, column
- `iloc` method allows you to choose index, column by numeric location (like a numpy array)

In [ ]:
df_ca['neuron0'] # test the type()

### Creating a new column

In [ ]:
df_ca['neuron46'] = df_ca['neuron0'] * 2
df_ca['neuron46']

In [ ]:
df_ca.columns

### Removing columns

In [ ]:
df_ca = df_ca.drop('neuron46', axis=1)
df_ca.columns

Can also drop rows this way:

In [ ]:
df_ca.drop(0.0, axis=0)

In [ ]:
df_ca[['neuron0', 'neuron1']]

### Selecting rows

In [ ]:
df_ca.loc[0:1.3] # reminder: loc selects columns and rows by their name/value (not by index)

### Selecting by row AND column

In [ ]:
df_ca.loc[0:0.8, 'neuron0':'neuron3']

**Notice slice INCLUDES stop index**

In [ ]:
df_ca.iloc[0:5] # reminder iloc selects columns/rows by their index

In [ ]:
df_ca.iloc[0:10, 0:3]

**Notice slicing with iloc EXCLUDES stop index**  
Just like numpy array slicing

<a id='conditional'></a>
### Conditional selection
- Boolean mask in bracket notation will select **indices**
- `isin` method is useful if checking if values are equal to one of multiple values

#### Boolean masks

- Provide boolean mask on index (rows) using bracket notation `[]`

In [ ]:
# in situ data # cz - up til now, seems like this can be done with neuron data and remove insitu data

insitu_ctrl = pd.read_csv(
    'sample_data/insitu-ctrl.txt',
    delimiter=' ',
    header=None,
).transpose()

insitu_fat = pd.read_csv(
    'sample_data/insitu-fat.txt',
    delimiter=' ',
    header=None,
).transpose()

for df in [insitu_ctrl, insitu_fat]:
    df.columns = ['nts', 'vgat', 'vglut2']
    df.columns.name = 'gene'
    df.index.name = 'cell'

# insitu_ctrl.columns = ['nts', 'vgat', 'vglut2']
# insitu_ctrl.columns.name = 'gene'
# insitu_ctrl.index.name = 'cell'

# insitu_fat.columns = ['nts', 'vgat', 'vglut2']
# insitu_fat.columns.name = 'gene'
# insitu_fat.index.name = 'cell'
    
insitu_ctrl

In [ ]:
insitu_ctrl.shape

In [ ]:
insitu_fat.shape

In [ ]:
list('abcde')

In [ ]:
df = pd.DataFrame(np.arange(20).reshape(4, 5), columns=list('abcde'))
df

In [ ]:
df[df['a'] > 7]

In [ ]:
insitu_ctrl[insitu_ctrl['nts'] > 245]

In [ ]:
insitu_ctrl[insitu_ctrl['nts'] > 245]['vgat']

In [ ]:
insitu_ctrl[insitu_ctrl['vglut2'] == 0]

For two conditions you can use `|` and `&` with parentheses.

In [ ]:
insitu_ctrl[(insitu_ctrl['nts'] > 60) & (insitu_ctrl['vgat'] > 60)]

#### `DataFrame.isin`

In [ ]:
gene_expr = pd.read_csv('sample_data/gene-expr.csv') # CZ 
gene_expr

In [ ]:
gaba_ix = gene_expr['gene'].isin(['Gad2', 'Gad1', 'Slc32a1'])
gaba_ix[10:20]

In [ ]:
gene_expr[gaba_ix]

---
<a id='index'></a>
## Pandas Index (and columns)
- Numpy array that act as labels for axes of DataFrames and Series
- Index object defines columns and indices (rows)

### Changing the index with method `reset_index` and `set_index`

In [ ]:
df_ca.head()

In [ ]:
# Remove current index
df_ca = df_ca.reset_index()
df_ca.head()

In [ ]:
new_ix = df_ca['time'] / 60  # `reset_index` moved the old index into a new column named 'index'
df_ca['time (m)'] = new_ix

In [ ]:
df_ca.head()

In [ ]:
# Set column 'time (m) as new index
df_ca = df_ca.set_index('time (m)')
df_ca.head()

---
<a id='mising'></a>
## Missing data and `nan`

- We've already seen how missing data poitns are filled with `nan`
- `nan` typically ignored by default (unlike numpy, remember `nanmean` and `nanmax`?)
- Can be easily removed with `dropna` method

In [ ]:
df_wts = pd.DataFrame({
    'mouse2': [29, 29, np.nan, 30, 29],
    'mouse5': [31, 30, np.nan, 30, 30],
    'mouse6': [33, 32, np.nan, np.nan, 33]
})

In [ ]:
df_wts

In [ ]:
df_wts.mean()

In [ ]:
df_wts.isnull()

In [ ]:
df_wts.dropna(axis=0, how='any') # remove rows if at least one column has nan

In [ ]:
df_wts.dropna(axis=0, how='all') # remove rows if all columns has nans

In [ ]:
df_wts.fillna(value=df_wts.mean()) # fill with mean value across rows for each column

In [ ]:
df_wts.interpolate(method='linear', axis=0) # linear interp

---
<a id='group'></a>
## `DataFrame.groupby` and `DataFrame.apply`

- Used to apply opeartions on subset of DataFrame

![groupby](https://i.stack.imgur.com/sgCn1.jpg)
stackoverflow.com

### Partitioning (grouping) data

In [ ]:
df = pd.read_csv('sample_data/gene-expr.csv')
df.head()

In [ ]:
df[df['cluster'] == 0]

In [ ]:
df.groupby('cluster').max()

In [ ]:
for x, y in df.groupby('cluster'): print(x, y)

In [ ]:
df.groupby('cluster').describe().head()

---
<a id='combine'></a>
## Merging, joining, and concatenating

- `concat` can do almost everything for you
- Other functions are `merge`, `join`, `append`

In [ ]:
# in situ data
insitu_ctrl = pd.read_csv('sample_data/insitu-ctrl.txt', delimiter=' ', header=None,).transpose()
insitu_fat = pd.read_csv('sample_data/insitu-fat.txt', delimiter=' ', header=None,).transpose()

# set column and index information
for df in [insitu_ctrl, insitu_fat]:
    df.columns = ['nts', 'vgat', 'vglut2']
    df.columns.name = 'gene'
    df.index.name = 'cell'
    
insitu_ctrl.head()

### Concatenation

- `concat` combines a list of DataFrames
- Works much like numpy--dimensions should match along the axis you are concatenating

In [ ]:
pd.concat([insitu_ctrl, insitu_fat], axis=1).head() # specify axis to concatenate along

In [ ]:
print('combined vertical length', len(insitu_ctrl)+len(insitu_fat))
pd.concat([insitu_ctrl, insitu_fat], axis=0).shape

In [ ]:
pd.concat([insitu_ctrl, insitu_fat], axis=0).sort_index().head()

**Notice Index labels are maintained and we have repeats**

**`ignore_index` resets index to default**

In [ ]:
pd.concat([insitu_ctrl, insitu_fat], axis=0, ignore_index=True).sort_index(axis=0).head()

In [ ]:
pd.concat([insitu_ctrl, insitu_fat], axis=0, ignore_index=True).sort_index(axis=0).tail()

In [ ]:
pd.concat([insitu_ctrl, insitu_fat], axis=1, ignore_index=True).head()

**Add hierarchical level to keep organization using parameter `keys`.**

In [ ]:
pd.concat([insitu_ctrl, insitu_fat], axis=0, keys=['ctrl', 'fat']).head()

In [ ]:
pd.concat([insitu_ctrl, insitu_fat], axis=0, keys=['ctrl', 'fat']).tail()

In [ ]:
pd.concat([insitu_ctrl, insitu_fat], axis=1, keys=['ctrl', 'fat']).describe()

## Operations

There are lots of operations with pandas that will be really useful to you, but don't fall into any distinct category. Let's show them here in this lecture:

In [ ]:
import pandas as pd
df = pd.DataFrame({'col1':[1,2,3,4],'col2':[444,555,666,444],'col3':['abc','def','ghi','xyz']})
df.head()

In [ ]:
df.info()

### Info on Unique Values

In [ ]:
df['col2'].unique()

In [ ]:
df['col2'].nunique()

In [ ]:
df['col2'].value_counts()

### Applying Functions

In [ ]:
def times2(x):
    return x*2

In [ ]:
df['col1'].apply(times2)

In [ ]:
df['col3'].apply(len)

** Permanently Removing a Column**

In [ ]:
del df['col1']

In [ ]:
df.columns

In [ ]:
df = pd.DataFrame({'col1':[1,2,3,4],'col2':[444,555,666,444],'col3':['abc','def','ghi','xyz']})

df.drop('col1', axis=1).columns

** Get column and index names: **

In [ ]:
df.columns

In [ ]:
df.index

** Sorting and Ordering a DataFrame:**

In [ ]:
df

In [ ]:
df.sort_values(by='col2')

## Data Input and Output

- Has a couple of `pd.read_XX` methods
- Can handle text files and Excel files

### CSV Input

In [ ]:
df = pd.read_csv('sample_data/gene-expr.csv')
df

In [ ]:
# df.to_csv('sample_data\example_saved.csv')

Pandas can read and write excel files, keep in mind, this only imports data. Not formulas or images, having images or macros may cause this read_excel method to crash. 

### Excel Input

In [ ]:
pd.read_excel('sample_data\Excel_Sample.xlsx',sheet_name='Sheet1')

### Excel Output

In [ ]:
df.to_excel('sample_data\Excel_Sample.xlsx',sheet_name='Sheet1')